In [1]:
%%time

#import boto3
import findspark
import os
#import pandas as pd

findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.pandas as ps

from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))

os.environ['PYARROW_IGNORE_TIMEZONE'] = str(1)

spark = SparkSession.builder \
    .appName("testing") \
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID']) \
    .config("spark.hadoop.fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY']) \
    .config("spark.hadoop.fs.s3a.proxy.host", "minio1") \
    .config("spark.hadoop.fs.s3a.endpoint", "minio1") \
    .config("spark.hadoop.fs.s3a.proxy.port", "9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

print("Spark inicializado com sucesso com as credenciais do S3")

Spark inicializado com sucesso com as credenciais do S3
CPU times: user 1.29 s, sys: 580 ms, total: 1.87 s
Wall time: 21.7 s


In [2]:
spark

In [3]:
df =  spark.read.csv('output.csv', header=True, inferSchema= True)   

In [4]:
df.show()

+---+--------------------+------------------+-----+------+------+-----+-------------------+------------------+-------------------+-------------------+------+------+------+------+-------------------+-------------------+
|_c0|                loss|   binary_accuracy|   TP|    TN|    FP|   FN|          precision|            recall|           val_loss|val_binary_accuracy|val_TP|val_TN|val_FP|val_FN|      val_precision|         val_recall|
+---+--------------------+------------------+-----+------+------+-----+-------------------+------------------+-------------------+-------------------+------+------+------+------+-------------------+-------------------+
|  0|0.001309059793129...|0.5363805890083313|376.0|1349.0|1392.0| 99.0|0.21266968548297882|0.7915789484977722| 0.7328429222106934| 0.4427083432674408|  31.0| 139.0| 161.0|  53.0| 0.1614583283662796|  0.369047611951828|
|  1|9.209519485011697E-4|0.7018035054206848|318.0|1939.0| 804.0|155.0|0.28342247009277344|0.6723044514656067| 0.62029546499

In [5]:
df.write.csv("s3a://raw/output2.csv")

In [5]:
teste = spark.read.csv("s3a://raw/estudante_escola.csv")

In [6]:
teste.write.parquet("s3a://silver/estudante_escola.parquet", mode="overwrite")

In [11]:
%%time

df =  spark.read.csv("s3a://raw/2018.csv", header=True)

CPU times: user 5 ms, sys: 861 µs, total: 5.86 ms
Wall time: 723 ms


In [12]:
df.show()

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|

In [2]:
%%time

df =  spark.read.parquet("s3a://silver/airplane-delay/2018")

CPU times: user 7.43 ms, sys: 1.18 ms, total: 8.62 ms
Wall time: 17.3 s


In [3]:
%%time

df =  spark.read.parquet("s3a://gold/airplane-delay")

CPU times: user 5.9 ms, sys: 499 µs, total: 6.4 ms
Wall time: 6.73 s


In [3]:
#contagem aproximada
df.sample(0.01).count()*100

4021100

In [4]:
%%time

small =  spark.read.parquet("s3a://gold/airplane-delay/ORIGIN=ABR")

CPU times: user 4.56 ms, sys: 128 µs, total: 4.69 ms
Wall time: 201 ms


In [9]:
#contagem aproximada
small.sample(0.01).count()*100

400

In [10]:
small.show()

+-------------------+----------+-----------------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+-----------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+-------------------+----------+-----------------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+--------

In [11]:
small.describe().show()

+-------+----------+-----------------+----+-----------------+-----------------+-----------------+------------------+------------------+------------------+----------------+------------------+------------------+------------------+--------------------+-----------------+--------+------------------+-------------------+-----------------+--------+------------------+------------------+-----------------+--------------------+-------------------+-----------+
|summary|OP_CARRIER|OP_CARRIER_FL_NUM|DEST|     CRS_DEP_TIME|         DEP_TIME|        DEP_DELAY|          TAXI_OUT|        WHEELS_OFF|         WHEELS_ON|         TAXI_IN|      CRS_ARR_TIME|          ARR_TIME|         ARR_DELAY|           CANCELLED|CANCELLATION_CODE|DIVERTED|  CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|         AIR_TIME|DISTANCE|     CARRIER_DELAY|     WEATHER_DELAY|        NAS_DELAY|      SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|
+-------+----------+-----------------+----+-----------------+-----------------+-----------------